In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing import Process, Manager, Pool
import numpy as np

### util method

In [2]:
def get_rating(id_trabalhador, escolaridade_trabalhador, graduacao_trabalhador, 
               cidade_trabalhador, escolaridade_vaga, graduacao_vaga, cidade_vaga, id_vaga):
    points = 0
    if escolaridade_trabalhador == escolaridade_vaga:
        points += 1
    if graduacao_trabalhador != np.nan and graduacao_trabalhador == graduacao_vaga:
        points += 1
    if cidade_trabalhador == cidade_vaga:
        points += 1
    
    return points
    #shared_list.append({'id_trabalhador': id_trabalhador, 'id_vaga':id_vaga, 'rating':rating})

## load trabalhadores

In [3]:
df_trabalhadores = pd.read_csv("data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_TRABALHADORES_SP.csv", sep=";", encoding="iso8859-1")

/home/chris/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_trabalhadores.dropna(subset=['PRETENSOES'], inplace=True)
df_trabalhadores[['PRETENSOES','LIXO']] = df_trabalhadores.PRETENSOES.str.split("(",1,expand=True)

In [5]:
#df_trabalhadores = df_trabalhadores[:30000]
df_trabalhadores = df_trabalhadores.sample(7000)
df_trabalhadores['id_trabalhador'] = df_trabalhadores.index
print(df_trabalhadores.shape)
df_trabalhadores.head()

(7000, 22)


,NACIONALIDADE,DEFICIENCIAS,BAIRRO,CEP,CODIGO_MUNICIPIO_IBGE,NOME_MUNICIPIO,UF,ESCOLARIDADE,ESTUDANTE,CURSOS_PROFISSIONALIZANTES,...,IDIOMAS,HABILITACAO,VEICULOS,DISP_VIAJAR,DISP_DORMIR_EMP,DISP_AUSENTAR_DOMIC,PRETENSOES,MUNICIPIOS_INTERESSE,LIXO,id_trabalhador
1268017,BRASILEIRA,NaN,VILA VELHA,1.88701e+07,351540,FARTURA,SP,Médio Completo,N,NaN,...,NaN,B,N,S,N,N,760305-ENCARREGADO DE CORTE NA CONFECCAO DO VE...,NaN,"N,83,0,Comercial)|760310-Encarregado de costur...",1268017
1221773,BRASILEIRA,NaN,VILA TERESINHA,2.85306e+06,355030,SAO PAULO,SP,Fundamental Completo,N,curso cuidador de idosos - ases,...,NaN,Nenhum,N,N,N,N,354130-Promotor de vendas especializado,NaN,"N,131,0,Indiferente)|516210-Cuidador de idosos...",1221773
2496794,BRASILEIRA,NaN,JARDIM SANTA INES II,1.22485e+07,354990,SAO JOSE DOS CAMPOS,SP,Médio Completo,N,NaN,...,NaN,Nenhum,N,N,N,N,731150-Montador de equipamentos eletrônicos,NaN,"N,486,0,Indiferente)",2496794
93731,BRASILEIRA,Membros Inferiores,JARDIM RUSSO,5.20504e+06,355030,SAO PAULO,SP,Superior Incompleto,S,NaN,...,NaN,Nenhum,N,N,N,N,252305-Auxiliar administrativo de diretoria,NaN,"N,0,0,Indiferente)|411005-Auxiliar de escritór...",93731
3141428,BRASILEIRA,NaN,PARQUE RESIDENCIAL ABILIO P,1.34832e+07,352690,LIMEIRA,SP,Médio Incompleto,N,NaN,...,NaN,Nenhum,N,S,N,N,413205-Atendente de agência,NaN,"N,13,0,Indiferente)|421105-ATENDENTE COMERCIAL...",3141428


## load vagas

In [6]:
df_vagas = pd.read_csv("data/vagas_mock.csv")
#df_vagas = df_vagas[:10000]
df_vagas = df_vagas.sample(3000)
print(df_vagas.shape)
df_vagas.head()

(3000, 7)


,cidade,escolaridade,graduacao,id_empresa,industria,nome_empresa,posicao
37759,SANTANA DE PARNAIBA,Fundamental Completo,Nenhum,d93af0f9-a8ab-4715-b35e-5639112e620e,Minerais,synthesize leading-edge infrastructures,391205-Inspetor de qualidade
60638,FRANCISCO MORATO,Médio Completo,Nenhum,55fd4744-b20f-4c07-ad27-332f1ebb580c,Farmacêutico,maximize best-of-breed methodologies,514110-Manobrista
58548,PEDERNEIRAS,Médio Completo,Nenhum,b3cb53ef-36ad-4999-ba8a-d22cb9a0e33e,Arte e Cultura,incentivize next-generation deliverables,517410-Porteiro de edifícios
66196,SAO JOSE DO RIO PRETO,Médio Completo,Nenhum,696a1e3f-b335-451d-b51e-e1d32c823fc0,Minerais,incubate interactive users,513205-Cozinheiro geral
58000,SANTANA DE PARNAIBA,Médio Completo,Nenhum,362be614-4d3c-40db-a0be-67b742002b14,Limpeza,innovate 24/7 action-items,782310-Motorista de furgão ou veículo similar


## Feature engineering
automaticamente "avalia" uma posicao para um trabalhador

In [7]:
rating_list = []
for index, row in tqdm(df_trabalhadores.iterrows(), total=df_trabalhadores.shape[0]):
    id_trabalhador = row['id_trabalhador']
    escolaridade_trabalhador = row['ESCOLARIDADE']
    graduacao_trabalhador = row['GRADUACOES']
    cidade_trabalhador = row['NOME_MUNICIPIO']
    
    for index_vaga, row_vaga in df_vagas.iterrows():
        escolaridade_vaga = row_vaga['escolaridade']
        graduacao_vaga = row_vaga['graduacao']
        cidade_vaga = row_vaga['cidade']
        id_vaga = row_vaga['id_empresa']
        
        
        rating = get_rating(id_trabalhador=id_trabalhador, escolaridade_trabalhador=escolaridade_trabalhador, graduacao_trabalhador=graduacao_trabalhador, 
               cidade_trabalhador=cidade_trabalhador, escolaridade_vaga=escolaridade_vaga, graduacao_vaga=graduacao_vaga, cidade_vaga=cidade_vaga, id_vaga=id_vaga)
        rating_list.append({'id_trabalhador':id_trabalhador, 'id_posicao':id_vaga, 'rating':rating})

In [8]:
del(df_trabalhadores)
del(df_vagas)

In [9]:
df_match = pd.DataFrame(rating_list)
print(df_match.shape)
df_match.head()

(21000000, 3)


,id_posicao,id_trabalhador,rating
0,d93af0f9-a8ab-4715-b35e-5639112e620e,1268017,0
1,55fd4744-b20f-4c07-ad27-332f1ebb580c,1268017,1
2,b3cb53ef-36ad-4999-ba8a-d22cb9a0e33e,1268017,1
3,696a1e3f-b335-451d-b51e-e1d32c823fc0,1268017,1
4,362be614-4d3c-40db-a0be-67b742002b14,1268017,1


In [10]:
del(rating_list)

In [11]:
df_match.rating.value_counts().plot.bar();

In [12]:
df_match.rating.value_counts()

0    13533545
1     7400962
2       65487
3           6
Name: rating, dtype: int64

In [13]:
df_match.to_csv("data/matches2.csv")